In [1]:
import pandas as pd
import numpy as np

## DB style DF joins  using merge

### Many to one merges

In [2]:
df1 = pd.DataFrame({
    "key" : ['b','b','a','c','a','a','b'],
    "data1" : range(7)
})
df2 = pd.DataFrame({
    "key" : ['a','b','d'],
    "data2" : range(3)
})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [3]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [4]:
pd.merge(df1,df2) 
# no col specified so overlapping taken

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [6]:
pd.merge(df1,df2 , on='key') # col on which joined
# inner join (NATURAL JOIN 1 col kept)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [8]:
# if col names different specify with left_on=, right_on=
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
 'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
'data2': range(3)})
print(df3)
print(df4)

  lkey  data1
0    b      0
1    b      1
2    a      2
3    c      3
4    a      4
5    a      5
6    b      6
  rkey  data2
0    a      0
1    b      1
2    d      2


In [10]:
# Equi INNER Join  kept both copies
pd.merge(df3,df4,left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [12]:
# how param , default 'inner'
# can be left / right / outer
# NAN -> float cols
pd.merge(df1,df2,how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


### Many to many merges (duplicatekeys on both side)

In [13]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
 'data1': range(6)})
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
 'data2': range(5)})
print(df1)
print(df2)

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   b      5
  key  data2
0   a      0
1   b      1
2   a      2
3   b      3
4   d      4


In [14]:
pd.merge(df1,df2,how='left',on='key')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [15]:
pd.merge(df1,df2,how='inner' , on='key')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [16]:
# multiple key merge on=[col list] same col names so one set only in result
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
 'key2': ['one', 'two', 'one'],
 'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
 'key2': ['one', 'one', 'one', 'two'],
 'rval': [4, 5, 6, 7]})
print(left)
print(right)


  key1 key2  lval
0  foo  one     1
1  foo  two     2
2  bar  one     3
  key1 key2  rval
0  foo  one     4
1  foo  one     5
2  bar  one     6
3  bar  two     7


In [17]:
pd.merge(left,right,how="outer" , on=["key1" , "key2"])

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [18]:
# index discarded when merged
#  issue : overlappin col names that are not join key
# suffixes added (_x,_y) default
pd.merge(left,right, on="key1")

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [21]:
pd.merge(left,right, on="key1" , suffixes=("_l","_r"))

,key1,key2_l,lval,key2_r,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


## Merging on Index 

In [22]:
# merge keys in index
# left_index=True or right_index=True or both in merge

In [24]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
 'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
print(left1)
print(right1)

  key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5
   group_val
a        3.5
b        7.0


In [25]:
pd.merge(left1,right1,left_on='key',right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [26]:
# outer
pd.merge(left1,right1,left_on='key',right_index=True,how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [27]:
# heirarchiacally indexed merge implicitly multikey join
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
 'Nevada', 'Nevada'],
 'key2': [2000, 2001, 2002, 2001, 2002],
 'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
 index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
 'Ohio', 'Ohio'],
 [2001, 2000, 2000, 2000, 2001, 2002]],
 columns=['event1', 'event2'])
print(lefth)
print(righth)

     key1  key2  data
0    Ohio  2000   0.0
1    Ohio  2001   1.0
2    Ohio  2002   2.0
3  Nevada  2001   3.0
4  Nevada  2002   4.0
             event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11


In [28]:
# kept index of left
pd.merge(lefth,righth,right_index=True, left_on=["key1" , "key2"])

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [29]:
pd.merge(lefth,righth,right_index=True, left_on=["key1" , "key2"], how="outer")


,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [31]:
# index on both side
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
 index=['a', 'c', 'e'],
 columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
 index=['b', 'c', 'd', 'e'],
 columns=['Missouri', 'Alabama'])
print(left2)
print(right2)

   Ohio  Nevada
a   1.0     2.0
c   3.0     4.0
e   5.0     6.0
   Missouri  Alabama
b       7.0      8.0
c       9.0     10.0
d      11.0     12.0
e      13.0     14.0


In [32]:
pd.merge(left2,right2,left_index=True,right_index=True)

,Ohio,Nevada,Missouri,Alabama
c,3.0,4.0,9.0,10.0
e,5.0,6.0,13.0,14.0


In [33]:
# outer
pd.merge(left2,right2,left_index=True,right_index=True, how="outer")

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [34]:
# join of df does by index
left2.join(right2, how="outer") 

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [38]:
# supports index of passed (right) merge with calling(left)'s col
left1.join(right1, on="key") # by default preserves left rows i.e left join 

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [39]:
# simple index-on-index merges, you can pass a list of DataFrames to join as aliter to concat func
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
 index=['a', 'c', 'e', 'f'],
  columns=['New York', 'Oregon'])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [41]:
left2.join([right1,another])

,Ohio,Nevada,group_val,New York,Oregon
a,1.0,2.0,3.5,7.0,8.0
c,3.0,4.0,NaN,9.0,10.0
e,5.0,6.0,NaN,11.0,12.0


In [42]:
left2.join([right1,another], how="outer")

,Ohio,Nevada,group_val,New York,Oregon
a,1.0,2.0,3.5,7.0,8.0
c,3.0,4.0,NaN,9.0,10.0
e,5.0,6.0,NaN,11.0,12.0
b,NaN,NaN,7.0,NaN,NaN
f,NaN,NaN,NaN,16.0,17.0
